# Group Assigment: G
### The Effect of Christmas on Flights

Alejandro Barrero, Patricia Cárcamo Acosta, Amala Eggers,  Luis Miguel Gallegos, Anna Koenig, Francisco Vigo

In [1]:
import findspark
findspark.init()

In [2]:
findspark.find()
import pyspark
findspark.find()

'/opt/spark-2.4.4-bin-hadoop2.7'

In [3]:
from pyspark import SparkContext, SparkConf
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

conf = pyspark.SparkConf().setAppName('appName').setMaster('local[4]')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

## Introduction to the Flights dataset

According to a 2010 report made by the US Federal Aviation Administration, the economic price of domestic flight delays entails a yearly cost of 32.9 billion dollars to passengers, airlines and other parts of the economy. More than half of that amount comes from passengers' pockets, as they do not only waste time waiting for their planes to leave, but also miss connecting flights, spend money on food and have to sleep on hotel rooms while they're stranded.

The report, focusing on data from year 2007, estimated that air transportation delays put a 4 billion dollar dent in the country's gross domestic product that year. Full report can be found 
<a href="http://www.isr.umd.edu/NEXTOR/pubs/TDI_Report_Final_10_18_10_V3.pdf">here</a>.

But which are the causes for these delays?

In order to answer this question, we are going to analyze the provided dataset, containing up to 1.936.758 different internal flights in the US for 2008 and their causes for delay, diversion and cancellation; if any.

The data comes from the U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics (BTS)

When initially downloading the December 2007 an comparing it with the January 2008 fligths dataset we previously worked with in class, we uncovered that there was a significant imbalance of the datasets. The January dataset only had 100,000 observations in comparison to 616,000 observations of the December 2007 dataset. Hence, we opted to download both datasets for December 2007 and January 2008 to complete our analysis.  The following are the features contained in our datasets:

1. **YEAR** 2007, 2008
2. **MONTH** 12, 1
3. **DAY_OF_MONTH** 1-31
4. **DAY_OF_WEEK** 1 (Monday) - 7 (Sunday)
5. **ORIGIN** origin IATA airport code
6. **DEST** destination IATA airport code
7. **DEP_DELAY** departure delay, in minutes
8. **DEP_DELAY_NEW** departure delay, in minutes
9. **DEP_DEL15** boolean representing 1 if departing flight was delayed over 15 minutes and 0 if it was not
10. **ARR_DELAY** arrival delay, in minutes: A flight is counted as "on time" if it operated less than 15 minutes later the scheduled time shown 
11. **ARR_DEL15** boolean representing 1 if arriving flight was delayed over 15 minutes and 0 if it was not 
12. **CANCELLED** *was the flight cancelled
13. **CANCELLATION_CODE** reason for cancellation (A = carrier, B = weather, C = NAS, D = security)
14. **DIVERTED** 1 = yes, 0 = no
15. **CARRIER_DELAY** in minutes: Carrier delay is within the control of the air carrier. Examples of occurrences that may determine carrier delay are: aircraft cleaning, aircraft damage, awaiting the arrival of connecting passengers or crew, baggage, bird strike, cargo loading, catering, computer, outage-carrier equipment, crew legality (pilot or attendant rest), damage by hazardous goods, engineering inspection, fueling, handling disabled passengers, late crew, lavatory servicing, maintenance, oversales, potable water servicing, removal of unruly passenger, slow boarding or seating, stowing carry-on baggage, weight and balance delays.
16. **WEATHER_DELAY** in minutes: Weather delay is caused by extreme or hazardous weather conditions that are forecasted or manifest themselves on point of departure, enroute, or on point of arrival.
17. **NAS_DELAY** in minutes: Delay that is within the control of the National Airspace System (NAS) may include: non-extreme weather conditions, airport operations, heavy traffic volume, air traffic control, etc.
18. **SECURITY_DELAY** in minutes: Security delay is caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas.
19. **LATE_AIRCRAFT_DELAY** in minutes: Arrival delay at an airport due to the late arrival of the same aircraft at a previous airport. The ripple effect of an earlier delay at downstream airports is referred to as delay propagation
20. **_c19** field automatically generated at data download. not used in the analysis

***

### Reading Data

We will be working with the flights datasets for December 2007 and January 2008 for our analysis on flight delays during the Holiday Season. First we will import the data and inspect the schema. 

In [4]:
flightsJan08 = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("Jan08.csv")

flightsDec07 = spark.read\
                .option("header", "true")\
                .option("inferSchema", "true")\
                .csv("Dec07.csv")

flightsJan08.printSchema()

flightsDec07.printSchema()

display(Markdown("The January 2008 DataFrame has **%d rows**." % flightsJan08.count()))
display(Markdown("The December 2007 DataFrame has **%d rows**." % flightsDec07.count()))

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- DEP_DELAY_NEW: double (nullable = true)
 |-- DEP_DEL15: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- ARR_DEL15: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- SECURITY_DELAY: double (nullable = true)
 |-- LATE_AIRCRAFT_DELAY: double (nullable = true)
 |-- _c19: string (nullable = true)

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (n

The January 2008 DataFrame has **605765 rows**.

The December 2007 DataFrame has **616382 rows**.

***

## Joining Datasets

Now, we will select the features that we are interested in analyzing and join the datasets. We inspect the new schema and the first 5 observations. The joint dataset has 1,222,147 rows! 

In [5]:
flights = flightsDec07.union(flightsJan08)

flights.printSchema()
flights.show(5)

display(Markdown("The joint flights DataFrame has **%d** rows." % flights.count()))

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- DEP_DELAY_NEW: double (nullable = true)
 |-- DEP_DEL15: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- ARR_DEL15: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- SECURITY_DELAY: double (nullable = true)
 |-- LATE_AIRCRAFT_DELAY: double (nullable = true)
 |-- _c19: string (nullable = true)

+----+-----+------------+-----------+------+----+---------+-------------+---------+---------+---------+---------+-----------------+--------+-------------

The joint flights DataFrame has **1222147** rows.

***

### Inspect Null Values
Our dataset contains Null Values in the features DEP_DELAY, DEP_DELAY_NEW, DEP_DEL15, ARR_DELAY, ARR_DEL15, CANCELLATION_CODE, CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY. We will see if these values actually represent flights with no delays or if they are real missing values from flight delay data.

In [6]:
#check nulls in date time columns, carrier columns, and delay information columns
flights.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) \
                for c in ['YEAR','MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK']]).show()

flights.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) \
                for c in ['ORIGIN', 'DEST', 'DEP_DELAY', 'DEP_DELAY_NEW',
                          'DEP_DEL15', 'ARR_DELAY', 'ARR_DEL15']]).show()

flights.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) \
                for c in ['CANCELLED', 'CANCELLATION_CODE', 'DIVERTED',
                          'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
                          'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']]).show()

+----+-----+------------+-----------+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|
+----+-----+------------+-----------+
|   0|    0|           0|          0|
+----+-----+------------+-----------+

+------+----+---------+-------------+---------+---------+---------+
|ORIGIN|DEST|DEP_DELAY|DEP_DELAY_NEW|DEP_DEL15|ARR_DELAY|ARR_DEL15|
+------+----+---------+-------------+---------+---------+---------+
|     0|   0|    38862|        38862|    38862|    41707|    41707|
+------+----+---------+-------------+---------+---------+---------+

+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+
|CANCELLED|CANCELLATION_CODE|DIVERTED|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+
|        0|          1183284|       0|       876581|       876581|   876581|        876581|             876581|
+---------+---------


## Your topic: the effect of Christmas

We want to check the different kinds of delays (at origin, due to weather, security, NAS, etc) and which airports are more prone to each of those. For that purpose, **you have to get the Flights dataset for December 2007**, having the same columns as the one you already have from January 2008. Make sure you get it from the right source so that the data are correct and comparable. You will use the January 2008 dataset as a baseline of the flights patterns in a normal month, and compare it with the patterns that show up in December (specially the last two weeks of the year). **Your answers should be restricted to four airports only: LAX (Los Angeles), SEA (Seattle-Tacoma), JFK (New York) and IAD (Washington)**. Once you have it, answer the following questions:

* Search for the day of the week of December the 24th and December the 31th 2007, and *count* the number of flights arriving  and departing to/from each airport. Compare it with the **average** number of flights arriving to each of those airports on the same day of the week during January 2008. How different was it? (Hint: January 2008 should have 3 or 4 weeks containing that day of the week, e.g. three Tuesdays, or three Wednesdays, etc, excluding New Year's Eve. Compute the average of them).
* Do you observe the same perceptual increase in the four airports, or is there an airport where the increase is much sharper than the others?
* What about the average arrival delay and departure delay? Repeat the analysis for both metrics (average delay of all the flights arriving on Dec 24, then average delay of flights arriving on Dec 31, and compare each of those averages with the average of the correspoding day of the week during the full month of January 2008).
* Do the same for December 25, 2007 and January 1, 2008. Search for the exact day of the week, and compare it with the average of the same day of the weeks of January 2008. Do you see the same increase or is it less sharp? Support your answer with data.

What insights can you provide after interpreting the results?

**HINT**: you can start [here](https://www.transtats.bts.gov/Fields.asp?Table_ID=236) to get the dataset. Click on `Data Tables` on top of the page and search for the period you want.

Your answers should be restricted to four airports only: LAX (Los Angeles), SEA (Seattle-Tacoma), JFK (New York) and IAD (Washington).

***

### Filter Airports of Interest : LAX, SEA, JFK, IAD

We will filter our main dataset and create separate datasets for Arrivals and Departures where our airports of interest - LAX, SEA, JFK, and IAD - are the Destination and Origin respectively. These are the datasets we will be working with. 

In [7]:
Arrivals = flights\
                .filter(flights.DEST.isin(['LAX', 'SEA', 'JFK', 'IAD']))\
                .cache()

Departures = flights\
                .filter(flights.ORIGIN.isin(['LAX', 'SEA', 'JFK', 'IAD']))\
                .cache()

In [8]:
Arrivals2007 = Arrivals\
                .filter("YEAR==2007")

Arrivals2008 = Arrivals\
                .filter("YEAR==2008")

ArrDelay2007 = Arrivals\
                .filter("ARR_DEL15==1")\
                .filter("YEAR==2007")

ArrDelay2008 = Arrivals\
                .filter("ARR_DEL15==1")\
                .filter("YEAR==2008")

Departures2007 = Departures\
                .filter("YEAR==2007")

Departures2008 = Departures\
                .filter("YEAR==2008")

DepDelay2007 = Departures\
                .filter("DEP_DEL15==1")\
                .filter("YEAR==2007")

DepDelay2008 = Departures\
                .filter("DEP_DEL15==1")\
                .filter("YEAR==2008")

***

### Flight Totals December 2007 - January 2008

Our analysis will examine the overall number of flights departing from and arriving to LAX, SEA, JFK, and IAD in December 2007 and January 2008. 
We discover that there are over 2,300 more flights arriving and departing from these airports in December 2007 than in January 2008. We will analyze arrivals and departures for both months paying special attention to the busiest travel dates and examining the impact of holidays on December 24, 25, 31 and January 1st. 

In [9]:
print("Total arrivals for December 2007 and January 2008:")
Arrivals.select("YEAR","MONTH","DEST")\
        .groupBy("YEAR","MONTH")\
        .count()\
        .show()

print("Total departures for December 2007 and January 2008:")
Departures.select("YEAR","MONTH","ORIGIN")\
          .groupBy("YEAR","MONTH")\
          .count()\
          .show()

ArrivalDifference = Arrivals2007.count() - Arrivals2008.count()
DepartureDifference = Departures2007.count() - Departures2008.count()

display(Markdown("The difference in arrivals between December 2007 and January 2008 is **%d**" % ArrivalDifference))
display(Markdown("The difference in departures between December 2007 and January 2008 is **%d**" % DepartureDifference))

Total arrivals for December 2007 and January 2008:
+----+-----+-----+
|YEAR|MONTH|count|
+----+-----+-----+
|2008|    1|44334|
|2007|   12|46651|
+----+-----+-----+

Total departures for December 2007 and January 2008:
+----+-----+-----+
|YEAR|MONTH|count|
+----+-----+-----+
|2008|    1|44297|
|2007|   12|46660|
+----+-----+-----+



The difference in arrivals between December 2007 and January 2008 is **2317**

The difference in departures between December 2007 and January 2008 is **2363**

***

## Arrivals


In [10]:
print("Total arrivals for December 2007 and January 2008 for each airport:")
Arrivals.select("YEAR","DEST")\
        .groupBy("DEST")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(4)

print("Total arrivals for the busiest days:")
Arrivals.select("YEAR","DAY_OF_MONTH")\
        .groupBy("DAY_OF_MONTH")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(7)

print("Flight arrivals by airport for the busiest days Dec 28 and Dec 21:")
Arrivals.filter("DAY_OF_MONTH==21 OR DAY_OF_MONTH==28")\
        .select("YEAR","DEST","DAY_OF_MONTH")\
        .groupBy("DEST","DAY_OF_MONTH")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(8)

Total arrivals for December 2007 and January 2008 for each airport:
+----+-----+-----+--------------+
|DEST| 2007| 2008|PercDifference|
+----+-----+-----+--------------+
| LAX|20222|18964|         -6.22|
| JFK|10613|10039|         -5.41|
| SEA| 8910| 8539|         -4.16|
| IAD| 6906| 6792|         -1.65|
+----+-----+-----+--------------+

Total arrivals for the busiest days:
+------------+----+----+--------------+
|DAY_OF_MONTH|2007|2008|PercDifference|
+------------+----+----+--------------+
|          21|1585|1448|         -8.64|
|          28|1584|1442|         -8.96|
|          20|1579|1369|         -13.3|
|          26|1579|1269|        -19.63|
|          27|1576|1372|        -12.94|
|          19|1570|1272|        -18.98|
|          14|1550|1449|         -6.52|
+------------+----+----+--------------+
only showing top 7 rows

Flight arrivals by airport for the busiest days Dec 28 and Dec 21:
+----+------------+----+----+--------------+
|DEST|DAY_OF_MONTH|2007|2008|PercDifference|


In December 2007 we had 46,651 total flight arrivals into LAX, SEA, JFK, and IAD, over 2,300 more than flight arrivals in January 2008. 

From the four airports, LAX has the highest volume of flights with 20,222 flights arriving in December 2007, almost double what JFK and SEA receive and almost three times more air traffic than IAD. When we look at the flight volume in December 2007 compared to January 2008, LAX had the most significant percentage decrease in flights (6.22%) compared to JFK, SEA and IAH. In contrast, IAD only saw a 1.65% decrease in flights from December to January. 

The days with most flights in 2007 are December 21, 28, 26, 20, 27 with over 1,576 flights arriving into the airports in our analysis. In comparison to the same date in January, we see a reduction of flights between 6.5% and 19.6%.

When we consider each airport's arrivals on those busiest travel dates, Dec 21 and 28, we see that LAX had 686 arrivals on each of those days, again significantly more than the rest of the airports. However, when looking at percentage differences between December 2007 and January 2008, the airport that saw the biggest flight arrival decline was JFK with a decrease of over 10%.

Let's now examine arrival delays. 


**Arrival Delays**

In [11]:
display(Markdown("There were a total of **%d** flight delays for our airports in December 2007." % ArrDelay2007.count()))
display(Markdown("There were a total of **%d** flight delays for our airports in January 2008." % ArrDelay2008.count()))

print("Total arrival delays for December 2007 and January 2008 for each airport:")
Arrivals.filter("ARR_DEL15==1")\
        .select("YEAR","DEST", "ARR_DELAY")\
        .groupBy("DEST")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(25)

print("Total arrival delays for the busiest days:")
Arrivals.filter("ARR_DEL15==1")\
        .select("YEAR","DAY_OF_MONTH","ARR_DELAY")\
        .groupBy("DAY_OF_MONTH")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(7)

print("Arrival delays by airport for the busiest days Dec 20, 21 and 22:")
Arrivals.filter("ARR_DEL15==1")\
        .filter("DAY_OF_MONTH==20 OR DAY_OF_MONTH==21 OR DAY_OF_MONTH==22")\
        .select("YEAR","DEST","DAY_OF_MONTH", "ARR_DELAY")\
        .groupBy("DEST","DAY_OF_MONTH")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(12)


There were a total of **15489** flight delays for our airports in December 2007.

There were a total of **11979** flight delays for our airports in January 2008.

Total arrival delays for December 2007 and January 2008 for each airport:
+----+----+----+--------------+
|DEST|2007|2008|PercDifference|
+----+----+----+--------------+
| LAX|6655|5886|        -11.56|
| JFK|3538|2356|        -33.41|
| SEA|3124|2140|         -31.5|
| IAD|2172|1597|        -26.47|
+----+----+----+--------------+

Total arrival delays for the busiest days:
+------------+----+----+--------------+
|DAY_OF_MONTH|2007|2008|PercDifference|
+------------+----+----+--------------+
|          21| 740| 491|        -33.65|
|          22| 687| 424|        -38.28|
|          20| 686| 239|        -65.16|
|          27| 677| 656|          -3.1|
|          28| 675| 438|        -35.11|
|          23| 673| 425|        -36.85|
|           2| 624| 618|         -0.96|
+------------+----+----+--------------+
only showing top 7 rows

Arrival delays by airport for the busiest days Dec 20, 21 and 22:
+----+------------+----+----+--------------+
|DEST|DAY_OF_MONTH|2007|2008|PercDifference|
+----

We consider those flights arriving more than 15 minutes late an arrival delay. In December 2007 our airports reported a total of 15,489 arrival delays in comparison to 11,979 arrival delays in January 2008. 

While LAX is again the airport with the highest volume of arrival delays, JFK had the highest percentage difference in delays between December 2007 and January 2008 (33.41%). 

The days with the highest numbers of arrival delays were December 20, 21, and 22 with over 686 flights arriving late in each day. On December 20, LAX had the highest number of flight arrival delays with 369. When comparing this figure to the same day in January 2008, LAX only had 101 delays, a drop of 72.6%. It is interesting to note, however, SEA on December 20 had 112 delays in comparison to 38 on January 20, an impressive 66% difference between both months. 

Let us focus on the reasons for arrival delay on both months, and check if the most frequent delay types are also the ones amounting to the greatest cumulative sum in minutes.


**Arrival Delays December 2007**

In [23]:
# Subset the dataframe with destination airports and delay types:
airportDelay07 = ArrDelay2007.select("DEST",
                                     "WEATHER_DELAY",
                                     "NAS_DELAY",
                                     "SECURITY_DELAY",
                                     "LATE_AIRCRAFT_DELAY")

# Group by destination airports:
groupedByAirport = airportDelay07.groupBy("DEST")

In [24]:
# Define the conditions for column selection based on maximum count or maximum cumulative total

condCount = "F.when" + \
            ".when".join(["(F.col('" + c + "') == F.col('MAX_COUNT'), '" + c + "')" \
                          for c in airportDelay07.columns[1:]])

condMins = "F.when" + \
           ".when".join(["(F.col('" + c + "') == F.col('MAX_MINUTES'), '" + c + "')" \
                         for c in airportDelay07.columns[1:]])

In [25]:
# Aggregate by counting the delay types and add column with maximum count 
# (by evaluating the appropriate condition):

print("Count of arrival delay types and maximum count for each airport December 2007:")
agg_byCount = groupedByAirport.agg(*[F.count(F.when(airportDelay07[c] != 0, c)).alias(c) \
                                     for c in airportDelay07.columns[1:]])

agg_byCount.withColumn("MAX_COUNT", F.greatest(*agg_byCount.columns[1:]))\
           .withColumn("MAX_COUNT_DELAY", eval(condCount))\
           .show()

#Aggregate by adding up the delay minutes per type and add column with maximum cumulative total 
#(by evaluating the appropriate condition):
print("Sum of arrival delay minutes per type for each airport December 2007:")
agg_byMins = groupedByAirport.agg(*[F.sum(airportDelay07[c]).alias(c) \
                                    for c in airportDelay07.columns[1:]])

agg_byMins.withColumn("MAX_MINUTES", F.greatest(*agg_byMins.columns[1:]))\
          .withColumn("MAX_MINUTES_DELAY", eval(condMins))\
          .show()

Count of delay types and maximum count for each airport December 2007:
+----+-------------+---------+--------------+-------------------+---------+-------------------+
|DEST|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|MAX_COUNT|    MAX_COUNT_DELAY|
+----+-------------+---------+--------------+-------------------+---------+-------------------+
| IAD|          129|      832|             3|               1123|     1123|LATE_AIRCRAFT_DELAY|
| SEA|          175|     2211|            39|               1429|     2211|          NAS_DELAY|
| LAX|          373|     4111|            50|               3301|     4111|          NAS_DELAY|
| JFK|          188|     2540|             6|               1623|     2540|          NAS_DELAY|
+----+-------------+---------+--------------+-------------------+---------+-------------------+

Sum of delay minutes per type for each airport December 2007:
+----+-------------+---------+--------------+-------------------+-----------+-------------------+


When we look at the reason of flight arrival delays in December 2007, we find and interesting mismatch for SEA and LAX between the most frequent reason and the reason that caused the longest delay in minutes. SEA and LAX had a total of 2,211 and 4,111 delays attributed to NAS_DELAYS respectively. However, NAS_DELAY only accounts for an aggregate of minute delays of 42,881 and 90,474 respectively, second to LATE_AIRCRAFT_ARRIVAL_DELAY with aggregate minute delays of 63,177 and 141,065. This means that while there were more NAS_DELAYS, these were shorter in duration than LATE_AIRCRAFT_DELAY which probably caused a greater inconvenience to travelers.

**Arrival Delays January 2008**

In [26]:
# Subset the dataframe with destination airports and delay types:
airportDelay08 = ArrDelay2008.select("DEST",
                                     "WEATHER_DELAY",
                                     "NAS_DELAY",
                                     "SECURITY_DELAY",
                                     "LATE_AIRCRAFT_DELAY")

# Group by destination airports:
groupedByAirport = airportDelay08.groupBy("DEST")

In [27]:
# Define the conditions for column selection based on maximum count or maximum cumulative total

condCount = "F.when" + \
            ".when".join(["(F.col('" + c + "') == F.col('MAX_COUNT'), '" + c + "')" \
                          for c in airportDelay08.columns[1:]])

condMins = "F.when" + \
           ".when".join(["(F.col('" + c + "') == F.col('MAX_MINUTES'), '" + c + "')" \
                         for c in airportDelay08.columns[1:]])

In [28]:
# Aggregate by counting the delay types and add column with maximum count 
# (by evaluating the appropriate condition):

print("Count of arrival delay types and maximum count for each airport January 2008:")
agg_byCount = groupedByAirport.agg(*[F.count(F.when(airportDelay08[c] != 0, c)).alias(c) \
                                     for c in airportDelay08.columns[1:]])

agg_byCount.withColumn("MAX_COUNT", F.greatest(*agg_byCount.columns[1:]))\
           .withColumn("MAX_COUNT_DELAY", eval(condCount))\
           .show()


# Aggregate by adding up the delay minutes per type and add column with maximum cumulative total 
# (by evaluating the appropriate condition):

print("Sum of arrival delay minutes per type for each airport January 2008:")
agg_byMins = groupedByAirport.agg(*[F.sum(airportDelay08[c]).alias(c) \
                                    for c in airportDelay08.columns[1:]])

agg_byMins.withColumn("MAX_MINUTES", F.greatest(*agg_byMins.columns[1:]))\
          .withColumn("MAX_MINUTES_DELAY", eval(condMins))\
          .show()

Count of delay types and maximum count for each airport January 2008:
+----+-------------+---------+--------------+-------------------+---------+-------------------+
|DEST|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|MAX_COUNT|    MAX_COUNT_DELAY|
+----+-------------+---------+--------------+-------------------+---------+-------------------+
| IAD|           53|      580|             6|                784|      784|LATE_AIRCRAFT_DELAY|
| SEA|          100|     1171|            21|               1103|     1171|          NAS_DELAY|
| LAX|          230|     3736|            37|               2926|     3736|          NAS_DELAY|
| JFK|          170|     1649|             7|                980|     1649|          NAS_DELAY|
+----+-------------+---------+--------------+-------------------+---------+-------------------+

Sum of delay minutes per type for each airport January 2008:
+----+-------------+---------+--------------+-------------------+-----------+-------------------+
|D

In January 2008 we find the same pattern for SEA and LAX between the reason with most number of delays (NAS_DELAY) versus the reason that caused the longest delay in minutes(LATE_AIRCRAFT_DELAY). It is also notable to mention that there is a reduction in both the number of delays and the minutes for each delay between December 2007 and January 2008. For example, JFK's most frequent reason for arrival delay  and reason for most delay minutes for both months was NAS_DELAY. However, in December 2007 it reported 2,540 arrival delays and 99,083 delay minutes compared to 1,649 delays and 45,212 delay minutes in January 2008.  

***

## Departures

In [29]:
print("Total departures for December 2007 and January 2008 for each airport:")
Departures.select("YEAR","ORIGIN")\
        .groupBy("ORIGIN")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(25)

print("Total departures for the busiest days:")
Departures.select("YEAR","DAY_OF_MONTH")\
        .groupBy("DAY_OF_MONTH")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(7)

print("Flight departures by airport for the busiest days :")
Departures.filter("DAY_OF_MONTH==20 OR DAY_OF_MONTH==21 OR DAY_OF_MONTH==28")\
        .select("YEAR","ORIGIN","DAY_OF_MONTH")\
        .groupBy("ORIGIN","DAY_OF_MONTH")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(12)

Total departures for December 2007 and January 2008 for each airport:
+------+-----+-----+--------------+
|ORIGIN| 2007| 2008|PercDifference|
+------+-----+-----+--------------+
|   LAX|20210|18945|         -6.26|
|   JFK|10628|10023|         -5.69|
|   SEA| 8913| 8543|         -4.15|
|   IAD| 6909| 6786|         -1.78|
+------+-----+-----+--------------+

Total departures for the busiest days:
+------------+----+----+--------------+
|DAY_OF_MONTH|2007|2008|PercDifference|
+------------+----+----+--------------+
|          21|1585|1443|         -8.96|
|          28|1582|1437|         -9.17|
|          20|1580|1372|        -13.16|
|          26|1577|1270|        -19.47|
|          27|1577|1372|         -13.0|
|          19|1570|1272|        -18.98|
|          18|1544|1455|         -5.76|
+------------+----+----+--------------+
only showing top 7 rows

Flight departures by airport for the busiest days :
+------+------------+----+----+--------------+
|ORIGIN|DAY_OF_MONTH|2007|2008|PercDif

Departures from our airports for the same period, December 2007 and January 2008, follow a similar pattern than our arrivals. 

In December 2007 we had 46,660 total flight departures from LAX, SEA, JFK, and IAD, and 44,297 flight departures in January 2008. 

LAX also leads all other airports in flight volume, reporting 20,210 departures in December 2007 and 18,945 in January 2008. This is double the traveler volume than the second largest airport, JFK. When considering the percentage decrease in flight departures between December and January, LAX saw the largest decrease of 6.26%.

The days with most flight departures are Dec 20, 21, and 28 with over 1,580 departures in each of these days. For comparative purposes, those same days in January had between 9% and 13% less flight departures. 

Now, we will examine flight departure delays and the reasons for those delays:

**Departure Delays**

In [30]:
display(Markdown("There were a total of **%d** departure delays for our airports in December 2007." % DepDelay2007.count()))
display(Markdown("There were a total of **%d** departure delays for our airports in January 2008." % DepDelay2008.count()))

print("Total departure delays for December 2007 and January 2008 for each airport:")
Departures.filter("DEP_DEL15==1")\
        .select("YEAR","ORIGIN", "DEP_DELAY")\
        .groupBy("ORIGIN")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(25)

print("Total departure delays for the busiest days:")
Departures.filter("DEP_DEL15==1")\
        .select("YEAR","DAY_OF_MONTH","DEP_DELAY")\
        .groupBy("DAY_OF_MONTH")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(7)

print("Departure delays by airport for the busiest days Dec 21, 21 and 23:")
Departures.filter("DEP_DEL15==1")\
        .filter("DAY_OF_MONTH==20 OR DAY_OF_MONTH==21 OR DAY_OF_MONTH==28")\
        .select("YEAR","ORIGIN","DAY_OF_MONTH", "DEP_DELAY")\
        .groupBy("ORIGIN","DAY_OF_MONTH")\
        .pivot("YEAR")\
        .count()\
        .withColumn("PercDifference", F.round(((F.col("2008")-F.col("2007"))/F.col("2007")*100),2))\
        .sort(F.col("2007").desc())\
        .show(12)

There were a total of **13073** departure delays for our airports in December 2007.

There were a total of **9976** departure delays for our airports in January 2008.

Total departure delays for December 2007 and January 2008 for each airport:
+------+----+----+--------------+
|ORIGIN|2007|2008|PercDifference|
+------+----+----+--------------+
|   LAX|5439|4782|        -12.08|
|   JFK|3009|1898|        -36.92|
|   SEA|2520|1805|        -28.37|
|   IAD|2105|1491|        -29.17|
+------+----+----+--------------+

Total departure delays for the busiest days:
+------------+----+----+--------------+
|DAY_OF_MONTH|2007|2008|PercDifference|
+------------+----+----+--------------+
|          21| 682| 437|        -35.92|
|          23| 667| 291|        -56.37|
|          22| 644| 325|        -49.53|
|          20| 632| 210|        -66.77|
|          27| 607| 500|        -17.63|
|          28| 576| 322|         -44.1|
|          19| 558| 212|        -62.01|
+------------+----+----+--------------+
only showing top 7 rows

Departure delays by airport for the busiest days Dec 21, 21 and 23:
+------+------------+----+----+--------------+
|ORIGIN|DAY_OF_MONTH|2007|

The total departure delays from our airports was 13,073 in December 2007  and 9,976 in January 2008. LAX had the highest number of depature delays on both time periods, reporting 5,439 departure delays in December 2007 and 4,782 in January 2008. However, JFK experienced the highest percentage change of almost 37% less flights in January than December. 

The days with most total number departure delays were December 21, 22 and 23, reporting over 640 delays per day. When examinging the percentage change between those days in December 2007 and January 2008, the days with most difference were December 19th and 20th with 62% and 66.8% respectively. 

On December 20th, LAX had 322 departure delays, compared to the same day in January where it only had 82 flight delays, an impressive difference of 74.5% 

Let us focus on the reasons for departure delay on both months, and check if the most frequent delay types are also the ones amounting to the greatest cumulative sum in minutes.

**Departure Delays December 2007**

In [31]:
# Subset the dataframe with origin airports and delay types:
airportDelay07 = DepDelay2007.select("ORIGIN",
                                     "WEATHER_DELAY",
                                     "NAS_DELAY",
                                     "SECURITY_DELAY",
                                     "LATE_AIRCRAFT_DELAY")

# Group by destination airports:
groupedByAirport = airportDelay07.groupBy("ORIGIN")

In [32]:
# Define the conditions for column selection based on maximum count or maximum cumulative total

condCount = "F.when" + \
            ".when".join(["(F.col('" + c + "') == F.col('MAX_COUNT'), '" + c + "')" \
                          for c in airportDelay07.columns[1:]])

condMins = "F.when" + \
           ".when".join(["(F.col('" + c + "') == F.col('MAX_MINUTES'), '" + c + "')" \
                         for c in airportDelay07.columns[1:]])

In [33]:
# Aggregate by counting the delay types and add column with maximum count 
# (by evaluating the appropriate condition):
print("Count of departure delay types and maximum count for each airport December 2007:")
agg_byCount = groupedByAirport.agg(*[F.count(F.when(airportDelay07[c] != 0, c)).alias(c) \
                                     for c in airportDelay07.columns[1:]])

agg_byCount.withColumn("MAX_COUNT", F.greatest(*agg_byCount.columns[1:]))\
           .withColumn("MAX_COUNT_DELAY", eval(condCount))\
           .show()

# Aggregate by adding up the delay minutes per type and add column with maximum cumulative total 
# (by evaluating the appropriate condition):
print("Sum of departure delay minutes per type for each airport December 2007:")
agg_byMins = groupedByAirport.agg(*[F.sum(airportDelay07[c]).alias(c) \
                                    for c in airportDelay07.columns[1:]])

agg_byMins.withColumn("MAX_MINUTES", F.greatest(*agg_byMins.columns[1:]))\
          .withColumn("MAX_MINUTES_DELAY", eval(condMins))\
          .show()

Count of departure delay types and maximum count for each airport December 2007:
+------+-------------+---------+--------------+-------------------+---------+-------------------+
|ORIGIN|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|MAX_COUNT|    MAX_COUNT_DELAY|
+------+-------------+---------+--------------+-------------------+---------+-------------------+
|   IAD|           77|      645|            23|                728|      728|LATE_AIRCRAFT_DELAY|
|   SEA|           89|      846|            14|               1197|     1197|LATE_AIRCRAFT_DELAY|
|   LAX|           70|     1979|            31|               2953|     2953|LATE_AIRCRAFT_DELAY|
|   JFK|          294|     1220|            10|               1350|     1350|LATE_AIRCRAFT_DELAY|
+------+-------------+---------+--------------+-------------------+---------+-------------------+

Sum of departure delay minutes per type for each airport December 2007:
+------+-------------+---------+--------------+---------------

When we examine the reason for delarture delays in December 2007, we find that the reason for both the most number of delays and most delay minutes is LATE_AIRCRAFT_DELAY. LAX and JFK are the airports with the highest number of delays, 2,952 and 1,350, and longest delays in minutes, 126,236 and 77,561 respectively. 

**Departure Delays January 2008**

In [37]:
# Subset the dataframe with origin airports and delay types:
airportDelay08 = DepDelay2008.select("ORIGIN",
                                     "WEATHER_DELAY",
                                     "NAS_DELAY",
                                     "SECURITY_DELAY",
                                     "LATE_AIRCRAFT_DELAY")

# Group by destination airports:
groupedByAirport = airportDelay08.groupBy("ORIGIN")

In [38]:
# Define the conditions for column selection based on maximum count or maximum cumulative total

condCount = "F.when" + \
            ".when".join(["(F.col('" + c + "') == F.col('MAX_COUNT'), '" + c + "')" \
                          for c in airportDelay08.columns[1:]])

condMins = "F.when" + \
           ".when".join(["(F.col('" + c + "') == F.col('MAX_MINUTES'), '" + c + "')" \
                         for c in airportDelay08.columns[1:]])

In [39]:
# Aggregate by counting the delay types and add column with maximum count 
# (by evaluating the appropriate condition):
print("Count of departure delay types and maximum count for each airport January 2008:")
agg_byCount = groupedByAirport.agg(*[F.count(F.when(airportDelay08[c] != 0, c)).alias(c) \
                                     for c in airportDelay08.columns[1:]])

agg_byCount.withColumn("MAX_COUNT", F.greatest(*agg_byCount.columns[1:]))\
           .withColumn("MAX_COUNT_DELAY", eval(condCount))\
           .show()

# Aggregate by adding up the delay minutes per type and add column with maximum cumulative total 
# (by evaluating the appropriate condition):
print("Sum of departure delay minutes per type for each airport January 2008:")
agg_byMins = groupedByAirport.agg(*[F.sum(airportDelay08[c]).alias(c) \
                                    for c in airportDelay08.columns[1:]])

agg_byMins.withColumn("MAX_MINUTES", F.greatest(*agg_byMins.columns[1:]))\
          .withColumn("MAX_MINUTES_DELAY", eval(condMins))\
          .show()

Count of departure delay types and maximum count for each airport January 2008:
+------+-------------+---------+--------------+-------------------+---------+-------------------+
|ORIGIN|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|MAX_COUNT|    MAX_COUNT_DELAY|
+------+-------------+---------+--------------+-------------------+---------+-------------------+
|   IAD|           85|      394|             3|                434|      434|LATE_AIRCRAFT_DELAY|
|   SEA|           40|      744|             8|                787|      787|LATE_AIRCRAFT_DELAY|
|   LAX|          131|     1625|            25|               2430|     2430|LATE_AIRCRAFT_DELAY|
|   JFK|          101|      757|            16|                730|      757|          NAS_DELAY|
+------+-------------+---------+--------------+-------------------+---------+-------------------+

Sum of departure delay minutes per type for each airport January 2008:
+------+-------------+---------+--------------+-----------------

In January 2008, all airports experienced the longest delays in minutes due to LATE_AIRCRAFT_ARRIVAL. For IAD, SEA and LAX the highest number of delays was also LATE_AIRCRAFT_ARRIVALS, but for JFK most delays were attributed to NAS_DELAY. 
Just like with arrival delays, we see that departure delays have a strong decline in both number of delays and length of delay in minutes. JFK saw reduction from December 2007 to January 2008 in the number of delays attributed to LATE_AIRCRAFT_DELAY from 1,350 to 730, and minutes from 77,561 to 33,774. 

***

## Christmas Eve and New Year's Eve

When analyzing the impact of important holidays in December 2008, we will consider the day of the week when Christmas Eve and New Years Eve took place and compare it with the average arrivals and departures for the same day of the week for the entire month of January 2008. In December 2007, these important holidays fell on a Monday. 

In [40]:
print("What day of the week did Christmas Eve and New Years Eve fall on in 2007?")
flights\
    .filter("YEAR==2007 and (DAY_OF_MONTH==24 or DAY_OF_MONTH==31)")\
    .select("DAY_OF_WEEK")\
    .show(1)

print("In 2007, Christmas Eve and New Years Eve fell on the 1st day of the week, Monday")

What day of the week did Christmas Eve and New Years Eve fall on in 2007?
+-----------+
|DAY_OF_WEEK|
+-----------+
|          1|
+-----------+
only showing top 1 row

In 2007, Christmas Eve and New Years Eve fell on the 1st day of the week, Monday


In [42]:
print("Let's see how many Mondays are in January 2008:")
flights\
    .filter("YEAR==2008 and DAY_OF_WEEK==1")\
    .select("DAY_OF_MONTH").distinct()\
    .show()

weeksJan = flights\
                .filter("YEAR==2008 and DAY_OF_WEEK==1")\
                .select("DAY_OF_MONTH")\
                .distinct()\
                .count()

display(Markdown("There are **%d** Mondays in January 2008." % weeksJan))

Let's see how many Mondays are in January 2008:
+------------+
|DAY_OF_MONTH|
+------------+
|          28|
|           7|
|          21|
|          14|
+------------+



There are **4** Mondays in January 2008.

**Arrivals and Arrival Delays December 24, 31 vs Mondays in January**

In [62]:
display(Markdown ("In December 2007, the total Arrivals are **%d**" %Arrivals2007.count()))

print("Total arrivals for December 24 and 31 for each airport:")
Arrivals.select("YEAR","DAY_OF_MONTH","DEST")\
        .filter("YEAR == 2007")\
        .filter("DAY_OF_MONTH == 24 or DAY_OF_MONTH == 31")\
        .groupBy("DEST")\
        .pivot("DAY_OF_MONTH")\
        .count()\
        .withColumn("AverageArrivals", F.round((F.col("24")+F.col("31"))/2,0))\
        .withColumn("ArrivalPercentage", F.round((F.col("24")+F.col("31"))/Arrivals2007.count()*100,0))\
        .sort(F.col("AverageArrivals").desc())\
        .show()

print("Total arrival delays for December 24 and 31 for each airport:")
Arrivals.select("YEAR","DAY_OF_MONTH","DEST")\
        .filter("ARR_DEL15==1")\
        .filter("YEAR == 2007")\
        .filter("DAY_OF_MONTH == 24 or DAY_OF_MONTH == 31")\
        .groupBy("DEST")\
        .pivot("DAY_OF_MONTH")\
        .count()\
        .withColumn("AverageArrivalDelays", F.round((F.col("24")+F.col("31"))/2,0))\
        .withColumn("ArrivalDelayPercentage", F.round((F.col("24")+F.col("31"))/ArrDelay2007.count()*100,0))\
        .sort(F.col("AverageArrivalDelays").desc())\
        .show()


display(Markdown ("In January 2008, the total Arrivals are **%d**" %Arrivals2008.count()))

print("Average flight arrivals for Mondays in January 2008:")
Arrivals.filter("YEAR==2008")\
        .filter("DAY_OF_WEEK==1")\
        .select("YEAR","DEST")\
        .sort("DEST", ascending=False)\
        .groupBy("DEST")\
        .count()\
        .withColumn("AverageArrivals", F.round(F.col("count")/weeksJan,0))\
        .withColumn("ArrPercentage", F.round(F.col("count")/Arrivals2008.count()*100,2))\
        .sort(F.col("AverageArrivals").desc())\
        .show()

print("Average late flight arrivals for Mondays in January 2008:")
Arrivals.filter("YEAR==2008")\
        .filter("DAY_OF_WEEK==1")\
        .filter("ARR_DEL15==1")\
        .select("YEAR","DEST","ARR_DELAY")\
        .groupBy("DEST")\
        .count()\
        .withColumn("AverageLateArrivals", F.round(F.col("count")/weeksJan,0))\
        .withColumn("LateArrPercentage", F.round(F.col("AverageLateArrivals")/ArrDelay2008.count()*100,2))\
        .sort(F.col("AverageLateArrivals").desc())\
        .show()


In December 2007, the total Arrivals are **46651**

Total arrivals for December 24 and 31 for each airport:
+----+---+---+---------------+-----------------+
|DEST| 24| 31|AverageArrivals|ArrivalPercentage|
+----+---+---+---------------+-----------------+
| LAX|606|608|          607.0|              3.0|
| JFK|349|348|          349.0|              1.0|
| SEA|276|285|          281.0|              1.0|
| IAD|212|210|          211.0|              1.0|
+----+---+---+---------------+-----------------+

Total arrival delays for December 24 and 31 for each airport:
+----+---+---+--------------------+----------------------+
|DEST| 24| 31|AverageArrivalDelays|ArrivalDelayPercentage|
+----+---+---+--------------------+----------------------+
| LAX| 84|154|               119.0|                   2.0|
| SEA| 54|100|                77.0|                   1.0|
| JFK| 84| 67|                76.0|                   1.0|
| IAD| 48| 52|                50.0|                   1.0|
+----+---+---+--------------------+----------------------+



In January 2008, the total Arrivals are **44334**

Average flight arrivals for Mondays in January 2008:
+----+-----+---------------+-------------+
|DEST|count|AverageArrivals|ArrPercentage|
+----+-----+---------------+-------------+
| LAX| 2488|          622.0|         5.61|
| JFK| 1326|          332.0|         2.99|
| SEA| 1115|          279.0|         2.51|
| IAD|  892|          223.0|         2.01|
+----+-----+---------------+-------------+

Average late flight arrivals for Mondays in January 2008:
+----+-----+-------------------+-----------------+
|DEST|count|AverageLateArrivals|LateArrPercentage|
+----+-----+-------------------+-----------------+
| LAX|  745|              186.0|             1.55|
| SEA|  305|               76.0|             0.63|
| JFK|  265|               66.0|             0.55|
| IAD|  190|               48.0|              0.4|
+----+-----+-------------------+-----------------+



**Arrival Delay Count and Cumulative Sum by Reason of Delay**

In [50]:
# Subset the dataframe with destination airports and delay types for the dates under study:
airportDelayChr07 = ArrDelay2007.select("DEST",
                                        "DAY_OF_MONTH",
                                        "WEATHER_DELAY",
                                        "NAS_DELAY",
                                        "SECURITY_DELAY",
                                        "LATE_AIRCRAFT_DELAY")\
                                .where("DAY_OF_MONTH==24 OR DAY_OF_MONTH==25 OR DAY_OF_MONTH==31")\

# Group by destination airports:
groupedByAirportDay = airportDelayChr07.groupBy("DEST","DAY_OF_MONTH")

In [51]:
# Define the conditions for column selection based on maximum count or maximum cumulative total

condCount = "F.when" + \
            ".when".join(["(F.col('" + c + "') == F.col('MAX_COUNT'), '" + c + "')" \
                          for c in airportDelayChr07.columns[1:]])

condMins = "F.when" + \
           ".when".join(["(F.col('" + c + "') == F.col('MAX_MINUTES'), '" + c + "')" \
                         for c in airportDelayChr07.columns[1:]])

In [52]:
agg_byCount = groupedByAirportDay.agg(*[F.count(F.when(airportDelayChr07[c] != 0, c)).alias(c) \
                                        for c in airportDelayChr07.columns[2:]])
print("Count of arrival delay types and maximum count for each airport December 24, 25, 31, 2007:")
agg_byCount.withColumn("MAX_COUNT", F.greatest(*agg_byCount.columns[2:]))\
           .withColumn("MAX_COUNT_DELAY", eval(condCount))\
           .orderBy(["DEST","DAY_OF_MONTH"],ascending = [True, True])\
           .show()

print("Sum of arrival delay minutes per type for each airport December 24, 25, 31, 2007:")
agg_byMins = groupedByAirportDay.agg(*[F.sum(airportDelayChr07[c]).alias(c) \
                                    for c in airportDelayChr07.columns[2:]])

agg_byMins.withColumn("MAX_MINUTES", F.greatest(*agg_byMins.columns[2:]))\
          .withColumn("MAX_MINUTES_DELAY", eval(condMins))\
          .orderBy(["DEST","DAY_OF_MONTH"],ascending = [True, True])\
          .show()

Count of arrival delay types and maximum count for each airport December 24, 25, 31, 2007:
+----+------------+-------------+---------+--------------+-------------------+---------+-------------------+
|DEST|DAY_OF_MONTH|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|MAX_COUNT|    MAX_COUNT_DELAY|
+----+------------+-------------+---------+--------------+-------------------+---------+-------------------+
| IAD|          24|            0|       11|             0|                 21|       21|LATE_AIRCRAFT_DELAY|
| IAD|          25|            4|       22|             0|                 23|       23|LATE_AIRCRAFT_DELAY|
| IAD|          31|            6|       15|             0|                 27|       27|LATE_AIRCRAFT_DELAY|
| JFK|          24|            0|       30|             0|                 48|       48|LATE_AIRCRAFT_DELAY|
| JFK|          25|            2|       24|             0|                 20|       24|          NAS_DELAY|
| JFK|          31|            4|    

When examining total arrivals per month, we find that there were a total of 46,651 arrivals in December 2007 and 44,334 arrivals in January 2008.

It is interesting to note the difference between flights arriving on December 24 and 31 to the average of flights arriving on Mondays in January 2008. On December 24 and 31 LAX had an average of 607 arrivals in comparison to 622 arrivals on average for Mondays in January 2008. When assessing percentages to total arrivals, December 24 and 31 saw an overall lower percentage, between 1% and 3%, compared to Mondays in January, between 2% and 5.6%

For flights with arrival delays, we only see a slight difference between the two periods. Our airports reported a low average of delays on Mondays in January between 0.4% and 1.55% to the total flight delays during January 2008. In contrast, December 24 and December 31 had flight arrival delays representing between 1% and 2% of total arrival delays for that month. For LAX this can be expressed as 119 average number of arrival delays for December 24 and 31 and 186 average number of arrival delays for January 2008. 

When we examine the number of arrival delays and the aggregate of arrival delay minutes for December 24, 25, and 31, we find that LAX on New Years Eve had 109 total arrival delays attributed to NAS_DELAY, yet the reason that caused most delays among our airports on these days was LATE_AIRCRAFT_DELAY. We can also determine that the reason that generated the longest delays on these important holidays in term of minutes was LATE_AIRCRAFT_DELAY for all airports except for SEA on Christmas Eve and LAX on New Year's Eve, whose longest delay in minutes was due to NAS_DELAY. 

**Departures and Departure Delays December 24, 31 VS Mondays in January**

In [48]:
display(Markdown ("In December 2007, the total Departures are **%d**" %Departures2007.count()))

print("Total departures for December 24 and 31 for each airport:")
Departures.select("YEAR","DAY_OF_MONTH","ORIGIN")\
        .filter("DAY_OF_MONTH == 24 or DAY_OF_MONTH == 31")\
        .groupBy("ORIGIN")\
        .pivot("DAY_OF_MONTH")\
        .count()\
        .withColumn("AverageDepartures", F.round((F.col("24")+F.col("31"))/2,0))\
        .withColumn("DeparturePercentage", F.round((F.col("24")+F.col("31"))/Departures2007.count()*100,0))\
        .sort(F.col("AverageDepartures").desc())\
        .show()

print("Total departure delays for December 24 and 31 for each airport:")
Departures.select("YEAR","DAY_OF_MONTH","ORIGIN")\
        .filter("DEP_DEL15==1")\
        .filter("DAY_OF_MONTH == 24 or DAY_OF_MONTH == 31")\
        .groupBy("ORIGIN")\
        .pivot("DAY_OF_MONTH")\
        .count()\
        .withColumn("AverageDepartureDelays", F.round((F.col("24")+F.col("31"))/2,0))\
        .withColumn("DepartureDelayPercentage", F.round((F.col("24")+F.col("31"))/ArrDelay2007.count()*100,0))\
        .sort(F.col("AverageDepartureDelays").desc())\
        .show()


display(Markdown ("In January 2008, the total Departures are **%d**" %Departures2008.count()))

print("Average flights departures for Mondays in January 2008:")
Departures.filter("YEAR==2008")\
        .filter("DAY_OF_WEEK==1")\
        .select("YEAR","ORIGIN")\
        .sort("ORIGIN", ascending=False)\
        .groupBy("ORIGIN")\
        .count()\
        .withColumn("AverageDepartures", F.round(F.col("count")/weeksJan,0))\
        .withColumn("DepPercentage", F.round(F.col("count")/Departures2008.count()*100,2))\
        .sort(F.col("AverageDepartures").desc())\
        .show()

print("Average departures delays for Mondays in January 2008:")
Departures.filter("YEAR==2008")\
        .filter("DAY_OF_WEEK==1")\
        .filter("DEP_DEL15==1")\
        .select("YEAR","ORIGIN","DEP_DELAY")\
        .groupBy("ORIGIN")\
        .count()\
        .withColumn("AverageLateDepartures", F.round(F.col("count")/weeksJan,0))\
        .withColumn("LateDepPercentage", F.round(F.col("AverageLateDepartures")/DepDelay2008.count()*100,2))\
        .sort(F.col("AverageLateDepartures").desc())\
        .show()

In December 2007, the total Departures are **46660**

Total departures for December 24 and 31 for each airport:
+------+----+----+-----------------+-------------------+
|ORIGIN|  24|  31|AverageDepartures|DeparturePercentage|
+------+----+----+-----------------+-------------------+
|   LAX|1214|1227|           1221.0|                5.0|
|   JFK| 674| 679|            677.0|                3.0|
|   SEA| 551| 559|            555.0|                2.0|
|   IAD| 429| 435|            432.0|                2.0|
+------+----+----+-----------------+-------------------+

Total departure delays for December 24 and 31 for each airport:
+------+---+---+----------------------+------------------------+
|ORIGIN| 24| 31|AverageDepartureDelays|DepartureDelayPercentage|
+------+---+---+----------------------+------------------------+
|   LAX|296|249|                 273.0|                     4.0|
|   JFK|160|108|                 134.0|                     2.0|
|   SEA| 77|144|                 111.0|                     1.0|
|   IAD| 96| 94|               

In January 2008, the total Departures are **44297**

Average flights departures for Mondays in January 2008:
+------+-----+-----------------+-------------+
|ORIGIN|count|AverageDepartures|DepPercentage|
+------+-----+-----------------+-------------+
|   LAX| 2476|            619.0|         5.59|
|   JFK| 1311|            328.0|         2.96|
|   SEA| 1115|            279.0|         2.52|
|   IAD|  891|            223.0|         2.01|
+------+-----+-----------------+-------------+

Average departures delays for Mondays in January 2008:
+------+-----+---------------------+-----------------+
|ORIGIN|count|AverageLateDepartures|LateDepPercentage|
+------+-----+---------------------+-----------------+
|   LAX|  648|                162.0|             1.62|
|   SEA|  265|                 66.0|             0.66|
|   JFK|  207|                 52.0|             0.52|
|   IAD|  166|                 42.0|             0.42|
+------+-----+---------------------+-----------------+



**Departure Delay Count and Cumulative Sum by Reason for Delay at Selected Dates**

In [54]:
# Subset the dataframe with origin airports and delay types for the dates under study:
airportDelayChr07 = DepDelay2007.select("ORIGIN",
                                        "DAY_OF_MONTH",
                                        "WEATHER_DELAY",
                                        "NAS_DELAY",
                                        "SECURITY_DELAY",
                                        "LATE_AIRCRAFT_DELAY")\
                                .where("DAY_OF_MONTH==24 OR DAY_OF_MONTH==25 OR DAY_OF_MONTH==31")\

# Group by destination airports:
groupedByAirportDay = airportDelayChr07.groupBy("ORIGIN","DAY_OF_MONTH")

In [55]:
# Define the conditions for column selection based on maximum count or maximum cumulative total

condCount = "F.when" + \
            ".when".join(["(F.col('" + c + "') == F.col('MAX_COUNT'), '" + c + "')" \
                          for c in airportDelayChr07.columns[1:]])

condMins = "F.when" + \
           ".when".join(["(F.col('" + c + "') == F.col('MAX_MINUTES'), '" + c + "')" \
                         for c in airportDelayChr07.columns[1:]])

In [56]:
agg_byCount = groupedByAirportDay.agg(*[F.count(F.when(airportDelayChr07[c] != 0, c)).alias(c) \
                                        for c in airportDelayChr07.columns[2:]])
print("Count of departure delay types and maximum count for each airport December 24, 25, 31, 2007:")
agg_byCount.withColumn("MAX_COUNT", F.greatest(*agg_byCount.columns[2:]))\
           .withColumn("MAX_COUNT_DELAY", eval(condCount))\
           .orderBy(["ORIGIN","DAY_OF_MONTH"],ascending = [True, True])\
           .show()

agg_byMins = groupedByAirportDay.agg(*[F.sum(airportDelayChr07[c]).alias(c) \
                                    for c in airportDelayChr07.columns[2:]])
print("Sum of departure delay minutes per type for each airport December 24, 25, 31, 2007:")
agg_byMins.withColumn("MAX_MINUTES", F.greatest(*agg_byMins.columns[2:]))\
          .withColumn("MAX_MINUTES_DELAY", eval(condMins))\
          .orderBy(["ORIGIN","DAY_OF_MONTH"],ascending = [True, True])\
          .show()

Count of departure delay types and maximum count for each airport December 24, 25, 31, 2007:
+------+------------+-------------+---------+--------------+-------------------+---------+-------------------+
|ORIGIN|DAY_OF_MONTH|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|MAX_COUNT|    MAX_COUNT_DELAY|
+------+------------+-------------+---------+--------------+-------------------+---------+-------------------+
|   IAD|          24|            0|        7|             2|                 13|       13|LATE_AIRCRAFT_DELAY|
|   IAD|          25|            0|       10|             0|                 14|       14|LATE_AIRCRAFT_DELAY|
|   IAD|          31|            2|       18|             0|                 13|       18|          NAS_DELAY|
|   JFK|          24|            2|       28|             0|                 40|       40|LATE_AIRCRAFT_DELAY|
|   JFK|          25|            0|        8|             0|                 14|       14|LATE_AIRCRAFT_DELAY|
|   JFK|          3

There were 44,297 total departures in January 2008, and total 46,660 departures in December 2007. 

In percentage terms, we see the flights on December 24 and 31 represent basically the same as the average of Mondays in January, between 2% and 5%. We do see a large difference in departures on December 24 and 31 for LAX, where on the stated holidays the average number of flights is 1,221 whereas the average for LAX on Mondays in January is 619. 

For departure delays, we can determine that December 24 and 31 see a significant higher number and percentage of flight departure delays, between 1% and 4%. As an example, LAX had 273 departure delays (4%) on Christmas Eve and New Year's Eve compared to 162 delays (1.62%) for Mondays in January 2008. 

When looking at the reason for departure delays in count and as an aggregate of delay in minutes, we see LATE_AIRCRAFT_DELAY is the top reason for departure delay and the reason that causes the longest delays in minutes. The only exception is for IAD and JFK who on New Year's Eve had more delays caused by NAS_DELAY, but for these airports on that day, the longest delay in minutes was LATE_AIRCRAFT_DELAY.

***

## December 25 and January 1st

Following the previous analysis, we will examine the difference in travel trends one day later: Christmas Day and New Year's Day. These holidays fell on a Tuesday, January had 5 Tuesdays.


In [57]:
print("What day of the week did Christmas Day 2007 and New Years Day 2008 fall on?")
flights\
    .filter("(YEAR==2007 and DAY_OF_MONTH==25) or (YEAR==2008 and DAY_OF_MONTH==1)")\
    .select("DAY_OF_WEEK")\
    .show(1)


print("December 25, 2007 and January 1, 2008 fell on the 2nd day of the week, Tuesday.")

What day of the week did Christmas Day 2007 and New Years Day 2008 fall on?
+-----------+
|DAY_OF_WEEK|
+-----------+
|          2|
+-----------+
only showing top 1 row

December 25, 2007 and January 1, 2008 fell on the 2nd day of the week, Tuesday.


In [59]:
print("Let's see how many Tuesdays are in January 2008:")
flights.filter("YEAR==2008 and DAY_OF_WEEK==2").select("DAY_OF_MONTH").distinct().show()
weeksJanTue = flights.filter("YEAR==2008 and DAY_OF_WEEK==2").select("DAY_OF_MONTH").distinct().count()

display(Markdown("There are **%d** Tuesdays in January 2008." % weeksJanTue))

Let's see how many Tuesdays are in January 2008:
+------------+
|DAY_OF_MONTH|
+------------+
|          22|
|           1|
|          15|
|           8|
|          29|
+------------+



There are **5** Tuesdays in January 2008.

**Arrivals and Arrival Delays December 25 and January 1 VS Tuesdays in January**

In [60]:
display(Markdown ("In December 2007, the total Arrivals are **%d**" %Arrivals2007.count()))

print("Total arrivals for December 25th and January 1st for each airport:")
Arrivals.select("YEAR","DAY_OF_MONTH","DEST")\
        .filter("(YEAR == 2007 and DAY_OF_MONTH == 25) or (YEAR == 2008 and DAY_OF_MONTH == 1)")\
        .groupBy("DEST")\
        .pivot("DAY_OF_MONTH")\
        .count()\
        .withColumn("AverageArrivals", F.round((F.col("25")+F.col("1"))/2,0))\
        .withColumn("ArrivalPercentage", F.round((F.col("25")+F.col("1"))/Arrivals2007.count()*100,0))\
        .sort(F.col("AverageArrivals").desc())\
        .show()

print("Total arrival delays for December 25 and 31 for each airport:")
Arrivals.select("YEAR","DAY_OF_MONTH","DEST")\
        .filter("ARR_DEL15==1")\
        .filter("(YEAR == 2007 and DAY_OF_MONTH == 25) or (YEAR == 2008 and DAY_OF_MONTH == 1)")\
        .groupBy("DEST")\
        .pivot("DAY_OF_MONTH")\
        .count()\
        .withColumn("AverageArrivalDelays", F.round((F.col("25")+F.col("1"))/2,0))\
        .withColumn("ArrivalDelayPercentage", F.round((F.col("25")+F.col("1"))/ArrDelay2007.count()*100,0))\
        .sort(F.col("AverageArrivalDelays").desc())\
        .show()

display(Markdown ("In January 2008, the total Arrivals are **%d**" %Arrivals2008.count()))

print("Average flight arrivals on Tuesdays in January 2008:")
Arrivals.filter("YEAR==2008")\
        .filter("DAY_OF_WEEK==2")\
        .select("YEAR","DEST")\
        .sort("DEST", ascending=False)\
        .groupBy("DEST")\
        .count()\
        .withColumn("AverageArrivals", F.round(F.col("count")/weeksJanTue,0))\
        .withColumn("ArrPercentage", F.round(F.col("count")/Arrivals2008.count()*100,2))\
        .sort(F.col("AverageArrivals").desc())\
        .show()

print("Average arrival delays on Tuesdays in January 2008:")
Arrivals.filter("YEAR==2008")\
        .filter("DAY_OF_WEEK==2")\
        .filter("ARR_DEL15==1")\
        .select("YEAR","DEST","ARR_DELAY")\
        .groupBy("DEST")\
        .count()\
        .withColumn("AverageLateArrivals", F.round(F.col("count")/weeksJanTue,0))\
        .withColumn("LateArrPercentage", F.round(F.col("AverageLateArrivals")/ArrDelay2008.count()*100,2))\
        .sort(F.col("AverageLateArrivals").desc())\
        .show()


In December 2007, the total Arrivals are **46651**

Total arrivals for December 25th and January 1st for each airport:
+----+---+---+---------------+-----------------+
|DEST|  1| 25|AverageArrivals|ArrivalPercentage|
+----+---+---+---------------+-----------------+
| LAX|646|615|          631.0|              3.0|
| JFK|347|339|          343.0|              1.0|
| SEA|288|270|          279.0|              1.0|
| IAD|219|200|          210.0|              1.0|
+----+---+---+---------------+-----------------+

Total arrival delays for December 25 and 31 for each airport:
+----+---+---+--------------------+----------------------+
|DEST|  1| 25|AverageArrivalDelays|ArrivalDelayPercentage|
+----+---+---+--------------------+----------------------+
| LAX|190|126|               158.0|                   2.0|
| JFK|118| 46|                82.0|                   1.0|
| SEA| 83| 67|                75.0|                   1.0|
| IAD| 72| 42|                57.0|                   1.0|
+----+---+---+--------------------+----------------------+



In January 2008, the total Arrivals are **44334**

Average flight arrivals on Tuesdays in January 2008:
+----+-----+---------------+-------------+
|DEST|count|AverageArrivals|ArrPercentage|
+----+-----+---------------+-------------+
| LAX| 3087|          617.0|         6.96|
| JFK| 1578|          316.0|         3.56|
| SEA| 1382|          276.0|         3.12|
| IAD| 1093|          219.0|         2.47|
+----+-----+---------------+-------------+

Average arrival delays on Tuesdays in January 2008:
+----+-----+-------------------+-----------------+
|DEST|count|AverageLateArrivals|LateArrPercentage|
+----+-----+-------------------+-----------------+
| LAX|  758|              152.0|             1.27|
| JFK|  383|               77.0|             0.64|
| SEA|  339|               68.0|             0.57|
| IAD|  258|               52.0|             0.43|
+----+-----+-------------------+-----------------+



The average arrivals for December 31, 2007 and January 1, 2008 range between 1% and 3%, whilst the average for Tuedays in January ranges between 2.5% and 7%. However, when we look at LAX, December 25 and January 1st saw an average of 631 flight arrivals compared to an average of 617 for Tuesdays in January 2008. The other aiports have minimal differences in the average number of flights for the period of this analysis.

For flights with arrival delays, our airports reported a similar average of delays on Tuesdays than our previous calculation of Mondays in January, reporting between 0.43% and 1.3% of flight delays. In comparison to December 25 and January 1, we only see an increase between 1% and 2% for those days. In terms of average number of flights, even LAX only sees a difference between December 25 and January 1 and the average of Tuesdays in January to be 158 and 152 respectively. 

**Departures and Departure Delays December 25, January 1 VS Tuesdays in January**

In [61]:
print("Total departures for December 25 and January 1 for each airport are:")
Departures.select("YEAR","DAY_OF_MONTH","ORIGIN")\
        .filter("(YEAR == 2007 and DAY_OF_MONTH == 25) or (YEAR == 2008 and DAY_OF_MONTH == 1)")\
        .groupBy("ORIGIN")\
        .pivot("DAY_OF_MONTH")\
        .count()\
        .withColumn("AverageDepartures", F.round((F.col("25")+F.col("1"))/2,0))\
        .withColumn("DeparturePercentage", F.round((F.col("25")+F.col("1"))/Departures2007.count()*100,0))\
        .sort(F.col("AverageDepartures").desc())\
        .show()

print("Total departure delays for December 25 and January 1 for each airport are:")
Departures.select("YEAR","DAY_OF_MONTH","ORIGIN")\
        .filter("DEP_DEL15==1")\
        .filter("(YEAR == 2007 and DAY_OF_MONTH == 25) or (YEAR == 2008 and DAY_OF_MONTH == 1)")\
        .groupBy("ORIGIN")\
        .pivot("DAY_OF_MONTH")\
        .count()\
        .withColumn("AverageDepartureDelays", F.round((F.col("25")+F.col("1"))/2,0))\
        .withColumn("DepartureDelayPercentage", F.round((F.col("25")+F.col("1"))/ArrDelay2007.count()*100,0))\
        .sort(F.col("AverageDepartureDelays").desc())\
        .show()


print("Average departures from our airports on Tuesdays in January 2008:")
Departures.filter("YEAR==2008")\
        .filter("DAY_OF_WEEK==2")\
        .select("YEAR","ORIGIN")\
        .sort("ORIGIN", ascending=False)\
        .groupBy("ORIGIN")\
        .count()\
        .withColumn("AverageDepartures", F.round(F.col("count")/weeksJanTue,0))\
        .withColumn("DepPercentage", F.round(F.col("count")/Departures2008.count()*100,2))\
        .sort(F.col("AverageDepartures").desc())\
        .show()

print("Average departure delays on Tuesdays in January:")
Departures.filter("YEAR==2008")\
        .filter("DAY_OF_WEEK==2")\
        .filter("DEP_DEL15==1")\
        .select("YEAR","ORIGIN","DEP_DELAY")\
        .groupBy("ORIGIN")\
        .count()\
        .withColumn("AverageLateDepartures", F.round(F.col("count")/weeksJanTue,0))\
        .withColumn("LateDepPercentage", F.round(F.col("AverageLateDepartures")/DepDelay2008.count()*100,2))\
        .sort(F.col("AverageLateDepartures").desc())\
        .show()

Total departures for December 25 and January 1 for each airport are:
+------+---+---+-----------------+-------------------+
|ORIGIN|  1| 25|AverageDepartures|DeparturePercentage|
+------+---+---+-----------------+-------------------+
|   LAX|646|624|            635.0|                3.0|
|   JFK|346|340|            343.0|                1.0|
|   SEA|293|272|            283.0|                1.0|
|   IAD|216|203|            210.0|                1.0|
+------+---+---+-----------------+-------------------+

Total departure delays for December 25 and January 1 for each airport are:
+------+---+---+----------------------+------------------------+
|ORIGIN|  1| 25|AverageDepartureDelays|DepartureDelayPercentage|
+------+---+---+----------------------+------------------------+
|   LAX|156|119|                 138.0|                     2.0|
|   JFK|120| 44|                  82.0|                     1.0|
|   SEA| 64| 65|                  65.0|                     1.0|
|   IAD| 66| 43|         

On average, LAX had close to 617 arrivals each Tuesday in January 2008. This represents close to 7% of total arrivals for the month of January. The airport with the lowest volume of travelers, IAD, only had an average of 219 arrivals each Monday of January 2008, representing 2.5% of all flight arrivals for the month. 

For flights with arrival delays, our airports reported a similar average of delays on Tuesdays than our previous calculation of Mondays in January, reporting between 0.43% and 1.3% of total flight delays during the month. 

Departures follow a similar trend as arrivals. On average, we see on average between 2.5% and 7% of departures taking place on Tuesdays in January 2008, slightly higher than our calculation for Mondays. This represents for LAX, the airport with most traffic, an average of 618 departures each Tuesday of the month. 

Departure delays from our airports represent between 0.41% and 1.25%, the highest number of delays belonging to LAX with 125 delays on average for Tuesdays in January 2008.

***

## Final Insights

When examining our analysis of arrival and departure differences between December 2007 and January 2008, we find a large difference in overall flight volume between both months. In particular, when looking at arrival delays and departure delays, we can clearly see a spike in number and percentages of delays due to the increase in the number of travelers passing through our airports during the holiday season. It is also important to note that, while LAX sees more flights than the other 3 airports in out analysis, JFK experiences the highest percentage change in general travel patterns. 

One of the key insights that we take from this analysis is that the busiest travel dates are not the major holidays: Christmas Eve, Christmas Day, New Year's Eve and New Year's Day. The highest volume of inbound and outbound flights occurs in the days preceeding these holidays, between December 18 - 22. Another important consideration is that January 2008, which in our analysis serves as our baseline month for comparing December 2007, also experiences an inflated number of travelers returning home after the holidays in the first days of the month.

When we examine the difference between December 24 and 31 and the average of Mondays in January 2008, we find minimal differences in arrivals, arrival delays, and departures. LAX is the airport with highest average difference between the two periods, with 607 arrivals in average for December 24 and December 31 against 622 average arrivals on Mondays in January 2008. The reported average departures for LAX during the same period is 1,221 and 619, showing that December 24 and 31 had double the average volume of flights departing when compared to the average of Monday flights in January 2008. On average, departure delays were also significant for LAX with an average of 273 delays for December 24 and 31 against and average of 162 for Mondays in January 2008. 

When we examine the difference one day later, between December 25 and January 1 and the average of Tuesdays in January 2008, we also find minimal differences in average arrivals, arrival delays, departures and departure delays. While LAX shows minimal differeces in average arrivals and arrival delays for the two periods in study, it shows higher average departures in December 25 and January 1 (635) against Tuesdays in January (618).  

When we examine the reasons for flight delays we found that, in general there are more delays and longer delays in minutes for both arrivals and departures in December 2007 than in January 2008. While the overwhelming reason for long delay in minutes in our analysis is LATE_AIRCRAFT_DELAY, we find that NAS_DELAY is a pressing reason as far as number of delays for many airports and dates in our analysis, but usually NAS_DELAYS are shorter in duration than LATE_AIRCRAFT_DELAY. 